In [2]:
import pandas as pd
import tensorflow as tf
import os
import shutil 

### Generate image data and add path's to it.

In [55]:
car_data = pd.read_csv("car_data.csv")
img_data = pd.read_csv("img_data.csv")

In [61]:
car_data = car_data.drop_duplicates('ID')
img_data = full_data.drop_duplicates('ID')

merged = pd.merge(car_data, img_data, on='ID', how='left')

In [62]:
def get_max_num(id):
    try:
        images = os.listdir("Myauto_data/Car_Images/Car_Images/{}".format(id))
    except:
        return 0
    return len(images)

In [63]:
merged['ID'] = merged.ID.apply(int)
merged['img_num'] = merged.ID.apply(get_max_num)

In [72]:
img_data = pd.DataFrame({'ID':[], 'Price ($)':[], 'Levy ($)':[], 'Manufacturer':[], 'Model':[], 'Prod. year':[],
       'Category':[], 'Leather interior':[], 'Fuel type':[], 'Engine volume':[], 'Mileage':[],
       'Cylinders':[], 'Gear box type':[], 'Drive wheels':[], 'Doors':[], 'Wheel':[], 'Color':[],
       'Interior color':[], 'Airbags':[], 'VIN':[], 'img_index':[], 'img_path':[]})

In [73]:
tmp_list = {'ID':[], 'Price ($)':[], 'Levy ($)':[], 'Manufacturer':[], 'Model':[], 'Prod. year':[],
       'Category':[], 'Leather interior':[], 'Fuel type':[], 'Engine volume':[], 'Mileage':[],
       'Cylinders':[], 'Gear box type':[], 'Drive wheels':[], 'Doors':[], 'Wheel':[], 'Color':[],
       'Interior color':[], 'Airbags':[], 'VIN':[], 'img_index':[], 'img_path':[]}

In [74]:
for i in merged.index:
    curr_row = merged.iloc[i]
    try:
        ls = [int(x) for x in curr_row.images.split('_')]
    except:
        try:
            ls = int(curr_row.images)
        except:
            continue
    for j in range(1, curr_row.img_num+1):
        for col in merged.columns[1:-2]:
            tmp_list[col].append(curr_row[col])
        tmp_list['ID'].append(str(curr_row['ID']))
        tmp_list['img_index'].append(str(j))
        tmp_list['img_path'].append('/' + str(curr_row.ID) + '/' + str(j) + '.jpg')

img_data = pd.DataFrame(tmp_list)

In [75]:
img_data.to_csv('generated/car_image_data.csv', index = False)

#### NOTE: is_car generated from vehicle_counting_tensorflow repo

### Generate Doors Data (35K)

In [118]:
df = pd.read_csv("generated/whole_data.csv")

In [119]:
cars_only = df[df["is_car"] == 1].sample(frac = 1)

In [120]:
cars_only.groupby("Doors").size()

Doors
2/3      9356
4/5    270968
>5       1286
dtype: int64

In [121]:
def select_column(df, column, condition):
    return df[df[column] == condition]

In [122]:
two_three = select_column(cars_only, "Doors", "2/3")[:9350]
four_five = select_column(cars_only, "Doors", "4/5")[:24400]
more_five = select_column(cars_only, "Doors", ">5")[:1250]

In [123]:
doors_data = pd.concat([two_three, four_five, more_five]).sample(frac = 1)
doors_data = final[["ID", "Doors", "img_path"]]

In [124]:
doors_data.to_csv('generated/doors_data.csv', index = False)

### Generate Colors Data (105K)

In [125]:
cars_only.groupby("Color").size()

Color
Beige             1946
Black            73758
Blue             21538
Brown             3583
Carnelian red     2145
Golden            1529
Green             4698
Grey             38408
Orange            1292
Pink               269
Purple             353
Red               9432
Silver           52951
Sky blue          1518
White            67139
Yellow             947
dtype: int64

In [126]:
black = select_column(cars_only, "Color", "Black")[:29100]
white = select_column(cars_only, "Color", "White")[:24900]
silver = select_column(cars_only, "Color", "Silver")[:19500]
grey = select_column(cars_only, "Color", "Grey")[:15000]
blue = select_column(cars_only, "Color", "Blue")[:10500]
red = select_column(cars_only, "Color", "Red")[:3900]
green = select_column(cars_only, "Color", "Green")[:2100]

In [127]:
color_data = pd.concat([black, white, silver, grey, blue, red, green]).sample(frac = 1)

In [128]:
color_data = color_data[['ID', 'Color', 'img_path']].sample(frac = 1)

In [129]:
color_data.groupby("Color").size().sum()

105000

In [130]:
color_data.to_csv('color_data.csv', index = False)

### Generate Category Data (50K)

In [131]:
cars_only.groupby('Category').size()

Category
Cabriolet         690
Coupe            7692
Goods wagon      1664
Hatchback       40778
Jeep            99430
Limousine          55
Microbus         1104
Minivan          5668
Pickup            515
Sedan          125635
Universal        3236
dtype: int64

In [132]:
sedan = select_column(cars_only, "Category", "Sedan")[:20000]
jeep = select_column(cars_only, "Category", "Jeep")[:15000]
hatchback = select_column(cars_only, "Category", "Hatchback")[:10000]
coupe = select_column(cars_only, "Category", "Coupe")[:5000]

In [133]:
category_data = pd.concat([sedan, jeep, hatchback, coupe]).sample(frac = 1)

In [134]:
category_data = category_data[["ID", "Category", "img_path"]]

In [135]:
category_data.to_csv('generated/category_data.csv', index = False)

# Split into Test and Train

In [141]:
doors_data = doors_data.sample(frac = 1)
test_size = int(len(doors_data)*2/10)
doors_test_data = doors_data[:test_size]
doors_train_data = doors_data[test_size:]
doors_test_data.to_csv('test/doors_test.csv', index = False)
doors_train_data.to_csv('train/doors_train.csv', index = False)

In [142]:
color_data = color_data.sample(frac = 1)
test_size = int(len(color_data)*1/10)
color_test_data = color_data[:test_size]
color_train_data = color_data[test_size:]
color_test_data.to_csv('test/color_test.csv', index = False)
color_train_data.to_csv('train/color_train.csv', index = False)

In [1]:
category_data = category_data.sample(frac = 1)
test_size = int(len(category_data)*1/10)
category_test_data = category_data[:test_size]
category_train_data = category_data[test_size:]
category_test_data.to_csv('test/category_test.csv', index = False)
category_train_data.to_csv('train/category_train.csv', index = False)

NameError: name 'category_data' is not defined

### Create image folders

In [17]:
os.mkdir("category_imgs")
os.mkdir("color_imgs")
os.mkdir("doors_imgs")

#### Create Category Data

In [18]:
category_data = pd.read_csv("train/category_train.csv")
for i in category_data.index:
    src_dir = "Myauto_data/Car_Images/Car_Images{}".format(category_data["img_path"][i])
    img = category_data["img_path"][i][1:].replace('/', "-")
    dst_dir = "category_imgs/{}".format(img)
    shutil.copy(src_dir, dst_dir)

In [19]:
category_data_test = pd.read_csv("test/category_test.csv")
for i in category_data_test.index:
    src_dir = "Myauto_data/Car_Images/Car_Images{}".format(category_data_test["img_path"][i])
    img = category_data_test["img_path"][i][1:].replace('/', "-")
    dst_dir = "category_imgs/{}".format(img)
    shutil.copy(src_dir, dst_dir)

#### Create Colors Data

In [21]:
color_data = pd.read_csv("train/color_train.csv")
for i in color_data.index:
    src_dir = "Myauto_data/Car_Images/Car_Images{}".format(color_data["img_path"][i])
    img = color_data["img_path"][i][1:].replace('/', "-")
    dst_dir = "color_imgs/{}".format(img)
    shutil.copy(src_dir, dst_dir)

In [22]:
color_data_test = pd.read_csv("test/color_test.csv")
for i in color_data_test.index:
    src_dir = "Myauto_data/Car_Images/Car_Images{}".format(color_data_test["img_path"][i])
    img = color_data_test["img_path"][i][1:].replace('/', "-")
    dst_dir = "color_imgs/{}".format(img)
    shutil.copy(src_dir, dst_dir)

#### Create Doors Data

In [24]:
doors_data = pd.read_csv("train/doors_train.csv")
for i in doors_data.index:
    src_dir = "Myauto_data/Car_Images/Car_Images{}".format(doors_data["img_path"][i])
    img = doors_data["img_path"][i][1:].replace('/', "-")
    dst_dir = "doors_imgs/{}".format(img)
    shutil.copy(src_dir, dst_dir)

In [26]:
doors_data_test = pd.read_csv("test/doors_test.csv")
for i in doors_data_test.index:
    src_dir = "Myauto_data/Car_Images/Car_Images{}".format(doors_data_test["img_path"][i])
    img = doors_data_test["img_path"][i][1:].replace('/', "-")
    dst_dir = "doors_imgs/{}".format(img)
    shutil.copy(src_dir, dst_dir)